In [2]:
!pip install jovian -q --upgrade

In [3]:
import torch
import jovian # cloud platform
import torch.nn as nn ## help us in creating and training of the neural network
import pandas as pd 
import matplotlib.pyplot as plt
import torch.nn.functional as F 
from torch.utils.data import DataLoader, TensorDataset, random_split ##need to review these 

<IPython.core.display.Javascript object>

In [4]:
data = "car_dataset.csv"

In [7]:
df_raw = pd.read_csv(data, delimiter=';')
df_raw.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [24]:
def customize_dataset(df_raw, random_str):
    df = df_raw.copy(deep = True) #copies the df
    df = df.sample(int(0.95*len(df)), random_state=int(ord(random_str[0]))) # random sample 
    df.Year = df.Year * ord(random_str[1])/100. #scale input?
    df.Selling_Price = df.Selling_Price * ord(random_str[2])/100. #scale target?
    #ord: converts a character into its Unicode code value.
    if ord(random_str[3]) % 2 == 1:
        df = df.drop(['Car_Name'], axis=1)
    return df

    

In [25]:
your_name = "deneme"
df = customize_dataset(df_raw, your_name)
df.head()
    

,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
199,2027.07,0.132,0.58,53000,Petrol,Individual,Manual,0
201,2030.10,3.575,6.79,58000,Diesel,Dealer,Manual,1
111,2036.16,1.265,1.50,8700,Petrol,Individual,Manual,0
125,2029.09,0.990,1.75,40000,Petrol,Individual,Manual,0
46,2033.13,2.915,4.89,64532,Petrol,Dealer,Manual,0


In [26]:
input_columns = ["Year","Present_Price","Kms_Driven","Owner"]
categorical_columns = ["Fuel_Type", "Seller_Type", "Transmission"]
output_columns = ["Selling_Price"]

## Data Preparation Step
### We will use Pytorch to predict car prices using machine learning

In [27]:
# Converting the dataset from dataframe to Pytorch Tensors:
# first convert it to Numpy Array;

def df_to_arrays(df):
    # copy the dataframe
    df_copy = df.copy(deep = True)
    #converting non-numeric categories to numeric
    for col in categorical_columns:
        df_copy[col] = df_copy[col].astype('category').cat.codes # every category assigned to a number
        
    #convert input and output to numpy arrays
    input_array = df_copy[input_columns].to_numpy()
    target_array = df_copy[output_columns].to_numpy()
    return input_array, target_array



input_array, target_array = df_to_arrays(df)

input_array, target_array
     
        
    
    
    
    
    
    
    
    
    
    

(array([[2.02707e+03, 5.80000e-01, 5.30000e+04, 0.00000e+00],
        [2.03010e+03, 6.79000e+00, 5.80000e+04, 1.00000e+00],
        [2.03616e+03, 1.50000e+00, 8.70000e+03, 0.00000e+00],
        ...,
        [2.03212e+03, 9.40000e+00, 7.10000e+04, 0.00000e+00],
        [2.03616e+03, 8.00000e-01, 2.00000e+04, 0.00000e+00],
        [2.03717e+03, 5.10000e-01, 4.30000e+03, 0.00000e+00]]),
 array([[ 0.132],
        [ 3.575],
        [ 1.265],
        [ 0.99 ],
        [ 2.915],
        [ 3.85 ],
        [ 6.6  ],
        [ 3.685],
        [ 0.528],
        [ 3.575],
        [ 0.418],
        [ 6.545],
        [ 7.645],
        [ 3.41 ],
        [ 1.595],
        [ 7.15 ],
        [ 2.75 ],
        [ 0.165],
        [10.175],
        [ 1.32 ],
        [ 0.88 ],
        [12.375],
        [ 4.4  ],
        [ 0.44 ],
        [ 7.15 ],
        [ 9.405],
        [ 0.495],
        [ 4.345],
        [ 0.561],
        [ 1.155],
        [ 7.92 ],
        [18.7  ],
        [ 4.95 ],
        [ 8.25 ],
 

In [28]:
# Now we have numpy arrays. We need to convert them to Pytorch Tensors
# and then we can use tensors to create a variable dataset.

inputs = torch.Tensor(input_array)
targets = torch.Tensor(target_array)

dataset = TensorDataset(inputs, targets)

train_ds, val_ds = random_split(dataset, [228, 57])
batch_size = 128

train_loader = DataLoader(train_ds, batch_size, shuffle = True)
val_loader = DataLoader(val_ds, batch_size)



## Creating the model

In [29]:
input_size = len(input_columns)
output_size = len(output_columns)
